# Redes neuronales hibridas  para clasificación multiple


In [ ]:
# Install the relevant packages.

#!pip install matplotlib torch torchvision torchaudio scikit-learn -q --extra-index-url https://download.pytorch.org/whl/cu118


In [ ]:
# importar las librerías necesarias
import cudaq
from cudaq import spin

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torchvision

from sklearn.model_selection import train_test_split

torch.manual_seed(22)
cudaq.set_random_seed(44)

In [ ]:
# Configurar el dispositivo
# Set CUDAQ and PyTorch to run on either CPU or GPU.

device = torch.device('cpu')
cudaq.set_target("qpp-cpu")

#cudaq.set_target("nvidia")
#device = torch.device("cuda:0")